Name : Rathnakar Rajanala

course : HDS-5230-07

Assignment : week 5 - Dask programming assignment



#For this assignment, will work on a dataset consisting of data aggregated from across the world, with data related to the spread of the COVID19 pandemic. Please complete the following tasks:
Obtain the dataset from here: https://coronadatascraper.com/timeseries.csv.zip. Details and some interesting work associated with it is found here: https://coronadatascraper.com/#home

Extract the .csv file from the zipped archive, and load it into a Dask dataframe.

In [107]:
import pandas as pd

In [109]:
import dask.dataframe as dd
import numpy as np

In [111]:
# Load the dataset with specific data type
csv_data = pd.read_csv(
    "timeseries.csv",
    dtype={
        'name': str,
        'level': str,
        'city': object,
        'county': str,
        'state': str,
        'country': str,
        'population': np.float64,
        'lat': float,
        'long': float,
        'url': str,
        'aggregate': object,
        'tz': str,
        'cases': np.float64,
        'deaths': np.float64,
        'recovered': np.float64,
        'active': np.float64,
        'tested': np.float64,
        'hospitalized': np.float64,
        'hospitalized_current': np.float64,
        'discharged': np.float64,
        'icu': np.float64,
        'icu_current': np.float64,
        'growthFactor': np.float64,
        'date': object
    }
)


In [113]:
# Convert  Pandas DataFrame to a Dask DATAFRAME
dask_df = dd.from_pandas(csv_data, npartitions=4)

# Filter for entries from the United State and group by state
us_data = dask_df[dask_df['country'] == 'United States']
state = us_data.groupby('state').size().compute()

#  result
print(state)

state
Alabama                          7068
Alaska                           1878
Arizona                          1823
Arkansas                         8085
California                       7006
Colorado                         6351
Connecticut                      1021
Delaware                          455
Florida                          7625
Georgia                         16593
Hawaii                            583
Idaho                            3610
Illinois                         9975
Nevada                           1518
Virginia                        13010
Washington                       4497
Indiana                          9720
Iowa                             9425
Kansas                           8211
Kentucky                        11419
Louisiana                        6915
Maine                            1785
Maryland                         2756
Massachusetts                    1820
Michigan                         8397
Minnesota                        8359
Missis

1.Create a new dataframe object that consists of samples (i.e., rows) corresponding to states in the US.

In [116]:
filtered_data = dask_df.query("country == 'United States'")
filtered_data = filtered_data.groupby("state").count()
filtered_data.compute()

,name,level,city,county,country,population,lat,long,url,aggregate,...,recovered,active,tested,hospitalized,hospitalized_current,discharged,icu,icu_current,growthFactor,date
state,,,,,,,,,,,,,,,,,,,,,
Alabama,7068,7068,0,6953,7068,7068,7068,7068,7068,7068,...,0,0,53,0,0,0,0,0,6997,7068
Alaska,1878,1878,0,1775,1878,1833,1833,1833,1878,1878,...,0,0,53,0,0,0,0,0,1844,1878
Arizona,1823,1823,0,1661,1823,1823,1823,1823,1823,1823,...,0,0,53,0,0,0,0,0,1792,1823
Arkansas,8085,8085,0,7968,8085,8085,8085,8085,8085,8085,...,4264,4376,5626,0,0,0,0,0,7529,8085
California,7006,7006,0,6843,7006,7006,7006,7006,7006,7006,...,3422,3052,4545,1436,1817,0,141,933,6589,7006
Colorado,6351,6351,0,6230,6351,6351,6351,6351,6351,6351,...,0,0,43,31,0,0,0,0,6285,6351
Connecticut,1021,1021,0,901,1021,1021,1021,1021,1021,1021,...,0,0,53,0,0,0,0,0,1012,1021
Delaware,455,455,0,338,455,455,455,455,455,455,...,0,207,53,0,0,0,0,0,451,455
Florida,7625,7625,0,7498,7625,7625,7625,7625,7625,7625,...,0,0,5229,0,0,0,0,0,7286,7625


The output is a new DataFrame that only includes U.S. states, eliminating territories such as Guam and Puerto Rico.  The DataFrame shows the total number of confirmed COVID-19 cases in each state.


 Texas had the highest case count, with 22,069 instances.
 Georgia has 16,593 cases.
 States such as California and Florida have approximately 7,000-7,600 instances.
 Smaller states, such as Delaware and Vermont, record substantially fewer incidents, approximately 455 and 1,625, respectively.
 This filtered DataFrame provides for a more concentrated examination of COVID-19's impact inside U.S. states, resulting in more relevant public health insights and regional comparisons. 


2.During the time period 2020-Jan-01 to 2021-Feb-28, rank states in terms of their per-capita mortality? Compute per-capita mortality during a specific period as the ratio of total deaths during that time period, to the average population of the state (compute the average population during the time period).

In [120]:
# Filter data and create 'seen' column as the sum of 'recovered' and 'active'
us_data = dask_df[dask_df['country'] == 'United States']
us_data['seen'] = us_data['recovered'] + us_data['active']

#  result
print(us_data.compute())

                                                     name   level  \
24958                Washoe County, Nevada, United States  county   
24959                Washoe County, Nevada, United States  county   
24960                Washoe County, Nevada, United States  county   
24961                Washoe County, Nevada, United States  county   
24962                Washoe County, Nevada, United States  county   
...                                                   ...     ...   
397823  Detroit City, Wayne County, Michigan, United S...    city   
397824  Detroit City, Wayne County, Michigan, United S...    city   
397825  Detroit City, Wayne County, Michigan, United S...    city   
397826  Detroit City, Wayne County, Michigan, United S...    city   
397827  Detroit City, Wayne County, Michigan, United S...    city   

                city         county     state        country  population  \
24958           <NA>  Washoe County    Nevada  United States    471519.0   
24959           <NA

In [122]:
# Convert 'date' column to datetime format
new_column_df['date'] = dd.to_datetime(new_column_df['date'])

# Filter data between January 2020 and February 2021
filtered_df = new_column_df[(new_column_df['date'] >= '2020-01-01') & (new_column_df['date'] <= '2021-02-28')]

# Group by 'state' and calculate total deaths and average population
state_summary = filtered_df.groupby('state').agg({
    'deaths': 'sum',
    'population': 'mean'
}).compute()

# Rename columns to understand
state_summary.rename(columns={'deaths': 'total_deaths', 'population': 'avg_population'}, inplace=True)

# Calculate per-capita mortality and rank of the states
state_summary['per_capita_mortality'] = state_summary['total_deaths'] / state_summary['avg_population']
state_summary['rank'] = state_summary['per_capita_mortality'].rank(ascending=False)

# show per-capita mortality and ranks sorted by rank
print(state_summary[['per_capita_mortality', 'rank']].sort_values('rank'))


                              per_capita_mortality  rank
state                                                   
New York                                  6.395701   1.0
Michigan                                  3.204753   2.0
Louisiana                                 2.735288   3.0
Illinois                                  2.043863   4.0
New Jersey                                2.031200   5.0
Georgia                                   2.026085   6.0
Pennsylvania                              1.831572   7.0
Virginia                                  1.359063   8.0
Mississippi                               1.356705   9.0
Indiana                                   1.353423  10.0
Ohio                                      1.058943  11.0
Iowa                                      0.958771  12.0
Massachusetts                             0.904775  13.0
Colorado                                  0.894296  14.0
Minnesota                                 0.768853  15.0
Kentucky                       

The ranking of US states based on per-capita COVID-19 mortality from January 1, 2020 to February 28, 2021 reveals significant diversity among them.

New York has the highest per-capita mortality rate (6.4), indicating that the pandemic has had a disproportionately severe impact on this state's population.
Michigan and Louisiana follow with per-capita mortality rates of 3.2 and 2.7, indicating that the pandemic had a considerable impact on them as well.
On the other end of the range, states like Hawaii (0.0017) and American Samoa (0.0) have very low per-capita fatality rates, indicating that they were less affected by the epidemic during this time.

This list emphasizes the disparities in the severity of COVID-19's impact between states, which may be influenced by factors such as healthcare infrastructure, population density, epidemic timing, and state regulations.

3.During the same time period, compute the case fatality rate (CFR) per month, using one of the approaches defined in this scientific brief from the World Health Organization: https://www.who.int/news-room/commentaries/detail/estimating-mortality-from-covid-19Links to an external site.. This computation should yield an array of dimensions 50 (states) X 14 (months). State the assumptions you are making in computing this metric.

In [126]:
import dask.dataframe as dd

# Convert 'date' column to datetime
new_column_df['date'] = dd.to_datetime(new_column_df['date'])

# Filter data for a specific time period
timeperiod_df = new_column_df[(new_column_df['date'] >= '2020-01-01') & (new_column_df['date'] <= '2021-02-28')]

# Extract year and month
timeperiod_df['year_month'] = timeperiod_df['date'].dt.strftime('%Y-%m')

# Group by 'state' and 'year_month' to compute total deaths and cases
monthly_stats = time_period_df.groupby(['state', 'year_month']).agg({
    'deaths': 'sum',
    'cases': 'sum'
}).compute()

# Calculate CFR
monthly_stats['cfr'] = (monthly_stats['deaths'] / monthly_stats['cases']) * 100

# Pivot the table to create CFR matrix
cfr_mat = monthly_stats.reset_index().pivot(index='state', columns='year_month', values='cfr')

# Display the CFR matrix
print(cfr_mat)

year_month                    2020-01   2020-02   2020-03    2020-04  \
state                                                                  
Alabama                           NaN       NaN  0.532313   2.830899   
Alaska                            NaN       NaN  0.335008   2.314519   
American Samoa                    NaN       NaN       NaN        NaN   
Arizona                           0.0  0.000000  0.000000   1.486545   
Arkansas                          NaN       NaN  0.915656   1.911450   
California                        0.0  0.000000  2.006735   3.479974   
Colorado                          NaN       NaN  0.939250   2.636616   
Connecticut                       NaN       NaN  1.814771   6.477626   
Delaware                          NaN       NaN  1.334107   2.734038   
Florida                           NaN       NaN  0.842669   2.905738   
Georgia                           NaN       NaN  2.185574   3.980977   
Guam                              NaN       NaN  2.702703   3.61

The case fatality rate (CFR) varies greatly by state and month.  In general, higher CFRs in the early months of 2020 (for example, April) indicate that some states saw higher fatality rates relative to confirmed cases, presumably due to overburdened healthcare systems.  CFRs often remained stable or declined with time, indicating improved healthcare responses and more effective testing.  States with consistently high CFRs may need additional public health initiatives.  Missing data (NaN) represents periods without any reported cases or fatalities.  This statistic aids in understanding how the pandemic has affected certain states.

4.Using this matrix as input, compute the ranking of states, on how the CFR rate changed over time. This computation will involve an aggregation of of month-to-month changes in CFR. Note that some of these month-to-month changes can be positive (CFR increases from a previous month to the current month) or negative (CFR decrease from a previous month to the current month), or zero (no change in CFR). You need to aggregate these individual changes, across all time periods.

In [130]:
import numpy as np
import pandas as pd

# Replace NaNs with 0 to avoid issues in diff and sum
cfr_array_clean = np.nan_to_num(cfr_mat)

# Compute month-to-month changes in CFR
cfr_changes = np.diff(cfr_array_clean, axis=1)

# Calculate total change for the each state
total_changes = np.sum(cfr_changes, axis=1)

# Create DataFrame for ranking
states = cfr_matrix.index.values  
rank_df = pd.DataFrame({
    'state': states,
    'total_change': total_changes
})

# Rank states based on total change (higher decrease = higher rank)
rank_df['rank'] = rank_df['total_change'].rank(ascending=False, method='min')

# Sort by rank
rank_df = rank_df.sort_values(by='rank').reset_index(drop=True)

# Display ranked states
print(rank_df)

                           state  total_change  rank
0                    Connecticut      9.500807   1.0
1                       Michigan      8.623360   2.0
2                     New Jersey      8.251406   3.0
3                  Massachusetts      7.437951   4.0
4                   Pennsylvania      7.418973   5.0
5                       New York      7.233961   6.0
6       Northern Mariana Islands      6.535948   7.0
7                  New Hampshire      6.436738   8.0
8   United States Virgin Islands      5.976096   9.0
9                   Rhode Island      5.821287  10.0
10              Washington, D.C.      5.330110  11.0
11                       Indiana      5.314359  12.0
12                          Ohio      5.275945  13.0
13                     Louisiana      5.138853  14.0
14                      Colorado      5.063691  15.0
15                      Illinois      4.802791  16.0
16                      Maryland      4.649307  17.0
17                       Vermont      4.559147

States with a greater total shift in CFR (such as Connecticut, Michigan, and New Jersey) saw considerable swings in death rates compared to case numbers, which could indicate problems such as overburdened healthcare systems or insufficient early interventions. On the other side, states with lower rankings (such as Hawaii and American Samoa) may have been more successful in controlling the epidemic or experienced fewer breakouts.

This rating is useful for identifying states that may have suffered the most severe public health consequences from COVID-19, which can inform policy changes, resource allocation, and future investigations into the performance of healthcare systems during the pandemic.

4.For each of the above operations, explain using appropriate reasoning, whether using a parallelized and/or distributed way of performing the computation makes sense.

Interpretation:-

Parallelization makes sense when working with massive data sets or needing to speed up computations. A smaller dataset (such as 50 states and 14 months) may not yield considerable improvements. However, if the dataset grows (e.g., more states, years, or granular data), parallel or distributed techniques may be useful in distributing the work among numerous processors or computers, boosting efficiency and reducing computing time.